In [1]:
import os
import psycopg2
import requests as req

from dotenv import load_dotenv
from psycopg2.extras import NamedTupleCursor

In [8]:
def get_latest_nct_ids(count):
    db_username = os.getenv("AACT_USERNAME")
    db_password = os.getenv("AACT_PASSWORD")

    if not db_username or not db_password:
        print("Please make sure AACT_USERNAME and AACT_PASSWORD env variables are set in .env")
        return

    conn = psycopg2.connect(
        host="aact-db.ctti-clinicaltrials.org",
        database="aact",
        user=db_username,
        password=db_password,
        port="5432",
        cursor_factory=NamedTupleCursor
    )
    
    query= f"""
    SELECT
        s.nct_id,
        s.brief_title,
        s.official_title,
        s.phase,
        s.overall_status,
        s.study_type,
        s.enrollment,
        s.results_first_posted_date,
        s.study_first_posted_date,
        s.last_update_posted_date,
        s.primary_completion_date,
        s.completion_date,
        sp.name AS sponsor_name,
        sp.lead_or_collaborator,
        sp.agency_class,
        bs.description AS brief_summary,
        dd.description AS detailed_description,
        c.name AS condition_name,
        i.name AS intervention_name,
        co.name AS country_name
    FROM
        studies s
    LEFT JOIN sponsors sp ON s.nct_id = sp.nct_id
    LEFT JOIN brief_summaries bs ON s.nct_id = bs.nct_id
    LEFT JOIN detailed_descriptions dd ON s.nct_id = dd.nct_id
    LEFT JOIN conditions c ON s.nct_id = c.nct_id
    LEFT JOIN interventions i ON s.nct_id = i.nct_id
    LEFT JOIN countries co ON s.nct_id = co.nct_id
    WHERE
        s.phase IN ('Phase 3')
        AND sp.name IN ('Pfizer')

    ORDER BY
        s.results_first_posted_date DESC NULLS LAST,
        s.last_update_posted_date DESC,
        s.primary_completion_date DESC,
        s.completion_date DESC
    LIMIT {count};
    """


    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
            nct_ids = [rec.nct_id for rec in curs.fetchall()]

    return list(set(nct_ids))

In [14]:
list_of_nct_id = get_latest_nct_ids(100_000)

In [15]:
len(list_of_nct_id)

751